In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import torch 
import vtk
import pyvista
pyvista.global_theme.notebook = True
pyvista.set_jupyter_backend('static')
# pyvista.set_jupyter_backend('trame')

seed = 160318
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
import time
from utils import *
from utils_training import *
import prepare_data
from utils_compare_methods import *
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.patches as mpatches
import pandas as pd
import gc
from utils_plot import * 
from scipy.spatial.distance import directed_hausdorff
from scipy import stats 

sns.set_theme("paper", rc={"xtick.bottom": True, "ytick.left": True}, font_scale=1.1)
colors = sns.color_palette("mako").as_hex()
my_cmap = sns.color_palette("viridis", as_cmap=True)

%load_ext autoreload
%autoreload 2


In [2]:
images_repo = "../images"
results_repo = "../results"

if not (os.path.exists(f"{images_repo}/")):
    os.makedirs(f"{images_repo}/")
if not (os.path.exists(f"{results_repo}/")):
    os.makedirs(f"{results_repo}/")

In [3]:
class MeshesConstructor:
    def __init__(self, params):
        self.params = params

    def create_mesh(self, mesh, cell_type, prune_z=False):
        cells = mesh.get_cells_type(cell_type)
        if prune_z:
            points = mesh.points[:, :2]
        else:
            points = mesh.points
        out_mesh = meshio.Mesh(points=points, cells={cell_type: cells})
        return out_mesh

    def create_standard_mesh(
        self, phi, hmax=0.05, plot_mesh_bool=False, return_times=False
    ):
        n = np.shape(phi)[0]
        M = square(n - 1, n - 1)
        M.debug = 4  # For debugging and mmg3d output

        # Setting a P1 level set function
        phi = phi.flatten()
        phiP1 = P1Function(M, phi)
        # Remesh according to the level set
        newM = mmg2d(
            M,
            hmin=hmax / 1.7,
            sol=phiP1,
            ls=True,
            verb=0,
            extra_args="-hsiz " + str(hmax / 1.41),
        )
        # Trunc the negative subdomain of the level set
        Mf = trunc(newM, 3)
        Mf.save("Thf_2.mesh")  # Saving in binary format

        in_mesh = meshio.read("Thf_2.mesh")
        out_mesh = self.create_mesh(in_mesh, "triangle", True)
        meshio.write("Thf_2.xdmf", out_mesh)

        with dolfinx.io.XDMFFile(MPI.COMM_WORLD, "Thf_2.xdmf", "r") as xdmf:
            mesh = xdmf.read_mesh(name="Grid")

        return mesh

    def main(self, i, size, nb_vert=64):
        params = self.params[i]
        mu0, mu1, sigma_x, sigma_y, amplitude, alpha, beta = params[:7]
        mu_x = params[7:10].reshape([1, 3, 1])
        mu_y = params[10:13].reshape([1, 3, 1])
        sigma_x_phi = params[13:16].reshape([1, 3, 1])
        sigma_y_phi = params[16:].reshape([1, 3, 1])

        xy = np.linspace(0.0, 1.0, 2 * nb_vert - 1)
        XX, YY = np.meshgrid(xy, xy)
        XX = XX.flatten()
        YY = YY.flatten()
        XXYY = np.stack([XX, YY])
        phi = call_phi(XXYY, mu_x, mu_y, sigma_x_phi, sigma_y_phi)
        phi = np.reshape(phi, (2 * nb_vert - 1, 2 * nb_vert - 1)).T
        mesh = self.create_standard_mesh(
            phi=phi, hmax=size, plot_mesh_bool=False, return_times=True
        )
        V = dolfinx.fem.functionspace(mesh, ("CG", degV))
        return V.tabulate_dof_coordinates()[:, :2]

    def construct_multiple_meshes(self, name="meshes_coords.npy"):
        list_coords = []
        for i in range(self.params.shape[0]):
            print(f"Mesh nb: {i+1} / {self.params.shape[0]}")
            coords = self.main(i, 0.0224, 64)
            list_coords.append(coords)

        nb_points = [len(list_coords[i]) for i in range(len(list_coords))]
        max_nb_points = max(nb_points)
        final_array = np.zeros((len(list_coords), max_nb_points, 2)) * np.nan
        print(f"{len(list_coords)=}")
        print(f"{max_nb_points=}")
        print(f"{final_array.shape=}")
        for i in range(final_array.shape[0]):
            final_array[i, : list_coords[i].shape[0], :] = list_coords[i]

        np.save(f"./{name}", final_array)
        return final_array


def hausdorff(a, b):
    return np.maximum(directed_hausdorff(a, b)[0], directed_hausdorff(b, a)[0])

In [ ]:
if not os.path.exists(f"{results_repo}/meshes_coords_training.npy"):
    params = np.load("../../data/params.npy")[:500]
    constructor = MeshesConstructor(params)
    final_array = constructor.construct_multiple_meshes(
        f"{results_repo}/meshes_coords_training.npy"
    )

if not os.path.exists(f"{results_repo}/meshes_coords_test.npy"):
    params = np.load("../../data_test/params.npy")[:]
    constructor = MeshesConstructor(params)
    final_array = constructor.construct_multiple_meshes(
        f"{results_repo}/meshes_coords_test.npy"
    )

In [ ]:
if not os.path.exists(f"{results_repo}/hausdorff_distances_training_test.npy"):
    final_array_training = np.load(f"{results_repo}/meshes_coords_training.npy")
    final_array_test = np.load(f"{results_repo}/meshes_coords_test.npy")
    list_meshes_coords_training = []
    for i in range(final_array_training.shape[0]):
        coords = final_array_training[
            i, np.where(~np.isnan(final_array_training[i, :, 0]))[0], :
        ]
        list_meshes_coords_training.append(coords)

    list_meshes_coords_test = []
    for i in range(final_array_test.shape[0]):
        coords = final_array_test[
            i, np.where(~np.isnan(final_array_test[i, :, 0]))[0], :
        ]
        list_meshes_coords_test.append(coords)

    hausdorff_distances_training_test = np.zeros((500, 300))
    for i in range(hausdorff_distances_training_test.shape[0]):
        for j in range(hausdorff_distances_training_test.shape[1]):
            hausdorff_distances_training_test[i, j] = hausdorff(
                list_meshes_coords_training[i], list_meshes_coords_test[j]
            )
    np.save(
        f"{results_repo}/hausdorff_distances_training_test.npy",
        hausdorff_distances_training_test,
    )

In [ ]:
hausdorff_distances_training_test = np.load(
    f"{results_repo}/hausdorff_distances_training_test.npy"
).T
minimal_distances = []
min_indices = []
for i in range(hausdorff_distances_training_test.shape[0]):
    minimal_distances.append(np.min(hausdorff_distances_training_test[i, :]))
    min_indices.append(np.argmin(hausdorff_distances_training_test[i, :]))

minimal_distances = np.array(minimal_distances)
indices = np.argsort(minimal_distances)
minimal_distances = minimal_distances[indices]
print(f"{np.min(minimal_distances)=:.3e}")
print(f"{np.max(minimal_distances)=:.3e}")
print(f"{np.median(minimal_distances)=:.3e}")

In [ ]:
phi_train = np.load("../../data/Phi.npy")
phi_test = np.load("../../data_test/Phi.npy")

coords_training = np.load(f"{results_repo}/meshes_coords_training.npy")
coords_test = np.load(f"{results_repo}/meshes_coords_test.npy")
list_meshes_coords_training = []
for i in range(coords_training.shape[0]):
    coords = coords_training[i, np.where(~np.isnan(coords_training[i, :, 0]))[0], :]
    list_meshes_coords_training.append(coords)

list_meshes_coords_test = []
for i in range(coords_test.shape[0]):
    coords = coords_test[i, np.where(~np.isnan(coords_test[i, :, 0]))[0], :]
    list_meshes_coords_test.append(coords)

# fig, axes = plt.subplots(2, 2, figsize=(6, 6), sharey=False)
plt.figure(figsize=(6, 6))
for ind, i in enumerate([0, 100, 200, 299]):
    plt.subplot(2, 2, ind + 1)
    tria = tri.Triangulation(
        list_meshes_coords_training[min_indices[indices[i]]][:, 0],
        list_meshes_coords_training[min_indices[indices[i]]][:, 1],
    )
    c = np.ones(tria.triangles.shape[0])
    cmap = "RdBu"
    plt.tripcolor(
        tria,
        c,
        alpha=0.5,
        lw=0,
        label="Training shape",
        antialiased=True,
        cmap=cmap,
    )

    tria = tri.Triangulation(
        list_meshes_coords_test[indices[i]][:, 0],
        list_meshes_coords_test[indices[i]][:, 1],
    )
    c = np.ones(tria.triangles.shape[0])
    cmap = "RdBu_r"

    plt.tripcolor(
        tria,
        c,
        alpha=0.5,
        lw=0,
        label="Test shape",
        antialiased=True,
        cmap=cmap,
    )
    if ind == 0 or ind == 1:
        plt.title(f"Hausdorff distance: {minimal_distances[i]:.3e}", fontsize=12)
    else:
        plt.title(
            f"Hausdorff distance: {minimal_distances[i]:.3e}", fontsize=12, y=-0.1
        )
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid()
plt.tight_layout()

colors = plt.colormaps["RdBu"]
red_patch = mpatches.Patch(color=colors(0), alpha=0.5, label="Training shape")
colors = plt.colormaps["RdBu_r"]
blue_patch = mpatches.Patch(color=colors(0), alpha=0.5, label="Test shape")

plt.legend(handles=[red_patch, blue_patch], loc=(-0.6, 1.0), ncol=2, fontsize=12)
plt.savefig(f"{images_repo}/mesh_hausdorff.png")
plt.show()

In [ ]:
L2_error_fno = np.load(f"{results_repo}/L2_error_fno.npy").flatten()[:][indices]
res = stats.linregress(minimal_distances, L2_error_fno)
plt.figure(figsize=(6, 6))
plt.plot(minimal_distances, L2_error_fno, ".")
plt.plot(
    minimal_distances,
    res.intercept + res.slope * minimal_distances,
    "r",
    label="Linear fitting",
)
# plt.xscale('log')
# plt.yscale('log')
plt.legend(fontsize=12)
plt.xlabel("Hausdorff distance to the closest training shape", fontsize=16)
plt.ylabel(r"$L^2$ relative error", fontsize=16)
plt.tight_layout()
plt.savefig(f"{images_repo}/error_vs_hausdorff.pdf")
plt.show()